# INFO 2950 Project Phase II

## Research Questions

The Kickstarter Dataset we found is based on the crowdfunding website, Kickstarter, which has helped fund many startups, products, and projects to become a reality. It is a combination of the publicly available information on past successful and unsuccessful kickstarter projects and data about the backers of each project. In this project, we are interested in exploring, what are the predictors for the success or failure of a Kickstarter campaign? How do kickstarter campaign success rates compare for US vs Non-US based projects? 
 


## Data Collection and Cleaning Data

Raw Data was collected by downloading the delimited data from the study at this link, https://www.icpsr.umich.edu/web/ICPSR/studies/38050/summary and Git Large Language Storage was used to upload the raw files onto Github. 

After opening the original public_data tsv file, we observed that several variables were not available publicly and were only available in the restricted dataset which we did not have access to. These variables include NAME, URL_NAME, and PROJECT_PAGE_LOCATION_COUNTRY.  In addition, there were some rows that had missing values, particularly in the PROJECT_PAGE_LOCATION_STATE, and PROJECT_PAGE_LOCATION_COUNTY columns which may indicate that these are user-inputs that are optional. In addition, although documentation for each dataset was provided upon download, descriptions for qualitative variables like STATE were unclear in how they categorized each startup into the 4 possible states (successful, failed, suspended, canceled). 

After opening a portion of the original backer_location tsv file, we observed that there were many rows that had missing values in either/both the USER_LOCATION_STATE and the USER_LOCATION_COUNTRY column. In addition, Numbers, the application tried to open the raw data in, warned us that it can only show the first 999999 rows of data. We either have to separate the original tsv into multiple tsv files to look at all of the data or do a boolean statement or GROUPBY to extract the most relevant information (Note: the first 182,253 entries have 50 people backing the project or more). Due to these issues, and being unsure of what to GROUPBY, we set this dataset aside for now to analyze the data in the first dataset (public_data.tsv).

In the future, we plan to merge the two tsv files by PID (Project ID???) to gain more insights on how location influences the amount of funding raised for each geographic location.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import duckdb

We imported the data and printed out the first 5 rows to view the columns provided by the dataset.

In [3]:
public_data = pd.read_csv('data/public_data.tsv', delimiter='\t')
public_data.head()

C:\Users\lovet\AppData\Local\Temp\ipykernel_24464\4156237419.py:1: DtypeWarning: Columns (9,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  public_data = pd.read_csv('data/public_data.tsv', delimiter='\t')


,CASEID,NAME,PID,CATEGORY,SUBCATEGORY,PROJECT_PAGE_LOCATION_NAME,PROJECT_PAGE_LOCATION_STATE,PROJECT_PAGE_LOCATION_COUNTY,PROJECT_PAGE_LOCATION_COUNTRY,UID,PROJECT_CURRENCY,GOAL_IN_ORIGINAL_CURRENCY,PLEDGED_IN_ORIGINAL_CURRENCY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,LAUNCHED_DATE,DEADLINE_DATE,STATE,URL_NAME
0,1,MASKED BY ICPSR,2137925650,11,301,London,England,Greater London,,1076478145,USD,"100,000,000",0,"$100,000,000",$0,0,8/11/2016,10/10/2016,canceled,MASKED BY ICPSR
1,2,MASKED BY ICPSR,1501531085,11,296,Los Angeles,CA,Los Angeles,,224946798,USD,"100,000,000",85,"$100,000,000",$85,4,12/19/2019,2/14/2020,canceled,MASKED BY ICPSR
2,3,MASKED BY ICPSR,953415668,16,51,Mexico,Baja California,Tijuana,,1772203542,MXN,"100,000,000",10,"$5,219,374",$1,1,3/1/2017,3/22/2017,failed,MASKED BY ICPSR
3,4,MASKED BY ICPSR,1371386304,18,18,Columbus,OH,Franklin,,1373465389,USD,"100,000,000",1,"$100,000,000",$1,1,6/4/2018,7/5/2018,canceled,MASKED BY ICPSR
4,5,MASKED BY ICPSR,1720842777,1,22,Toronto,ON,Toronto,,1455666383,CAD,"100,000,000",0,"$80,610,122",$0,0,5/1/2015,6/30/2015,failed,MASKED BY ICPSR


We checked for duplicates in the public_data.tsv dataset by selecting the rows that appeared more than once according to the unique PID. Since there are no rows that appeared more than once, we know that there is no duplicate PIDs in the dataset.

In [4]:
duckdb.sql("SELECT PID, COUNT(PID) FROM public_data GROUP BY PID HAVING COUNT(PID) > 1").df()

,PID,count(PID)


We dropped the following columns: NAME, PROJECT_PAGE_LOCATION_COUNTRY, and URL_NAME since they contained restricted information that wasn't available to the public. We also dropped the UID, and the rows that are canceled or suspended.

In [21]:
public_data_df = pd.DataFrame(public_data)
clean_data = public_data_df.drop(columns = ["NAME", "PROJECT_PAGE_LOCATION_COUNTRY", "URL_NAME", "UID"])

clean_data = duckdb.sql("SELECT * FROM clean_data WHERE STATE = 'successful' OR STATE = 'failed'").df()

clean_data["GOAL_IN_USD"] = clean_data["GOAL_IN_USD"].replace({'$': '', ',': ''}, regex = True)
clean_data.head()

,CASEID,PID,CATEGORY,SUBCATEGORY,PROJECT_PAGE_LOCATION_NAME,PROJECT_PAGE_LOCATION_STATE,PROJECT_PAGE_LOCATION_COUNTY,PROJECT_CURRENCY,GOAL_IN_ORIGINAL_CURRENCY,PLEDGED_IN_ORIGINAL_CURRENCY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,LAUNCHED_DATE,DEADLINE_DATE,STATE
0,56,128197700,12,35,Osaka,Ishikawa Prefecture,Hakui-gun,JPY,"68,000,000","81,030,744",$623968,"$743,538",12613,3/26/2018,5/5/2018,successful
1,63,1900597540,12,35,Tokyo,Tokyo Prefecture,,JPY,"53,808,516","481,621,841",$510735,"$4,571,420",46307,7/27/2020,8/29/2020,successful
2,83,1012629901,12,35,Ishikawa,Okinawa Prefecture,,JPY,"45,000,000","98,863,825",$414327,"$910,266",8051,6/3/2019,6/11/2019,successful
3,175,596722349,12,35,Chiba-shi,Chiba Prefecture,,JPY,"16,665,000","23,606,140",$148285,"$210,047",1705,9/20/2018,10/20/2018,successful
4,184,2056420743,16,337,Tokyo,Tokyo Prefecture,,JPY,"15,000,000","17,406,300",$133229,"$154,601",656,10/7/2017,12/6/2017,successful


We created different datasets for successful campaigns and failed campaigns

In [6]:
success_proj = duckdb.sql("SELECT * FROM clean_data WHERE STATE = 'successful'").df()
print(success_proj.head())
failed_proj = duckdb.sql("SELECT * FROM clean_data WHERE STATE = 'failed'").df()
print(failed_proj.head())

   CASEID         PID  CATEGORY  SUBCATEGORY PROJECT_PAGE_LOCATION_NAME  \
0      56   128197700        12           35                      Osaka   
1      63  1900597540        12           35                      Tokyo   
2      83  1012629901        12           35                   Ishikawa   
3     175   596722349        12           35                  Chiba-shi   
4     184  2056420743        16          337                      Tokyo   

  PROJECT_PAGE_LOCATION_STATE PROJECT_PAGE_LOCATION_COUNTY PROJECT_CURRENCY  \
0         Ishikawa Prefecture                    Hakui-gun              JPY   
1            Tokyo Prefecture                                           JPY   
2          Okinawa Prefecture                                           JPY   
3            Chiba Prefecture                                           JPY   
4            Tokyo Prefecture                                           JPY   

  GOAL_IN_ORIGINAL_CURRENCY PLEDGED_IN_ORIGINAL_CURRENCY GOAL_IN_USD  \
0 

We also divided the dataset based on the median campagin goals for successful and failed data.

In [10]:
success_goal_median = success_proj["GOAL_IN_USD"].median()
print(success_goal_median)
failed_goal_median = failed_proj["GOAL_IN_USD"].median()
print(failed_goal_median)

TypeError: Cannot convert ['$623,968' '$510,735' '$414,327' ... '$500' '$10,000' '$1,000'] to numeric

## Exploratory Data Analysis

We wanted to explore the relationship between the goal in USD and the number of backers between a successful project compared with a failed project. 

In [9]:
plt.scatter(success_proj["GOAL_IN_USD"], success_proj["BACKERS_COUNT"])
plt.xlabel("Campaign Goal (USD)")
plt.ylabel("Number of Backers")

Text(0, 0.5, 'Number of Backers')

Error in callback <function flush_figures at 0x00000209088E6C00> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

## Data Description

The attributes (columns) of the dataset are the PID (Project ID), Category (numbers indicating what topic the project falls under), Subcategory (numbers indicating the subtopics within each category), the Project Location Name, State, County, and Country. Other relevant attributes included are the Project Currency, the Project’s Original Goal (how much money the creators were aiming for), amount of money pledged, the number of backers, the launched date, deadline date, and state. State is defined as either suspended (funding for a project is stopped when the project violates Kickstarter’s rules), canceled (the creators of a project dropped the project themselves), failed (the project didn’t meet funding goals), and successful (project meets funding goals). Each observation (row) corresponds to one unique project, indexed with a CaseID.

This dataset was created by Jonathan Leland, Kickstarter’s Head of Insights for a cross-sectional study on the different creative projects that seek crowdfunding. He created the dataset to observe data on the projects from Kickstarter. The creation of this dataset is funded by Kickstarter Data, the company crowdfunding platform that directs the Kickstarter Project. The study is distributed through the Inter-University Consortium for Political and Social Research (ICPSR). 

Some columns may be presented as optional for users to fill out, so we’re left with missing values in the data. For example, for the PROJECT_LOCATION_STATE, PROJECT_LOCATION_COUNTY, PROJECT_LOCATION_COUNTRY column, there are some missing data values. But there are no missing values in the PROJECT_LOCATION_NAME, meaning that the project creators were required to put down the location name for their project. Several restrictions were placed on the data before releasing it for public use. The names of the people/groups who created past Kickstarter projects (NAME) and the links to their projects’ websites (URL_NAME), and the descriptions of their projects (BLURB) are masked in the public data set. This ensures anonymity of survey participants and prevents disclosure risk from survey participants.

This is a link to a google drive folder containing the raw source data: https://drive.google.com/drive/folders/1gySTPJShuvqEASlYBR5odxKNP2Wk9uca?usp=sharing

## Data Limitations

There are several limitations to our research. Since many variables on our public data tsv are restricted, including the project description, we can’t draw a clear relationship between what exactly about a specific category that leads to it being more or less successful than other categories. This affects the meaning of our results because there are unclear causes that may have a greater impact on how successful a project is. Thus, We can only rely on the Category and Subcategory columns, which only shows numbers corresponding to the category type, but not the names of the categories themselves. So, we don't actually have a clear view on the project content. 
There may be outside factors such as branding, social media presence, and popularity of the project before it was put on Kickstarter that may affect the success of the project. These confounding variables can skew our results. 
Furthermore, there are some unclear phrases not explained by the dataset documentation, such as PID and UID. We made the assumption that PID means Project ID, and used it to merge the two datasets. However, if it actually is something else, then it can negatively affect our data. 


## Questions For Reviewers